# Import and sandbox

In [1]:
import numpy as np
import pandas as pd
import torch 


In [3]:
df = pd.read_parquet('/projects/cpjump1/jump/load_data/ctrls_images.parquet')

In [ ]:
df.columns

In [15]:
images_cols = [
 'FileName_OrigAGP',
 'FileName_OrigDNA',
 'FileName_OrigER',
 'FileName_OrigMito', 
 'FileName_OrigRNA']

In [22]:
metadata = df.iloc[7][images_cols].to_dict()


In [ ]:
metadata

In [ ]:
for img_path in df.iloc[7][images_cols].values:
    print(img_path)

In [ ]:
pd.DataFrame(df.iloc[7][images_cols].values, columns=["zs"])

In [ ]:
pd.read_csv('/projects/synsight/repos/phenoseeker/tmp/metadata.csv')

In [ ]:
np.load('/projects/synsight/repos/phenoseeker/tmp/extracted_features.npy')

# check n_wells 

## Imports

In [2]:
df_phenom = pd.read_parquet('/projects/synsight/data/openphenom/raw_well_embeddings/wells_em_openphenom.parquet')

In [3]:
df_dinov2_s = pd.read_parquet('/projects/synsight/data/dinov2_small/raw_well_embeddings/wells_em_dinov2s.parquet')

In [4]:
df_dinov2_g = pd.read_parquet('/projects/synsight/data/dinov2/raw_well_embeddings/wells_em_dinov2.parquet')

In [5]:
df_resnet = pd.read_parquet('/projects/cpjump1/jump/images_embeddings/resnet50_raw/metadata/load_wells/df_meta_raw_well.parquet')

In [6]:
df_chada = pd.read_parquet('/projects/synsight/data/chada/well_embeddings/metadata_raw_mean.parquet')

In [7]:
df_chada_2 = pd.read_parquet('/projects/synsight/data/chada/chada_jump.parquet')

In [8]:
df_meta = pd.read_csv('/projects/cpjump1/jump/metadata/complete_metadata.csv')

/tmp/ipykernel_3390879/3787900309.py:1: DtypeWarning: Columns (2,8,9,10,11,12,13,14,15,18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_meta = pd.read_csv('/projects/cpjump1/jump/metadata/complete_metadata.csv')


## Reference

In [17]:
images_df_path = "/projects/cpjump1/jump/load_data/final"
images_df = pd.read_parquet(images_df_path)

images_df["Metadata_Batch"] = images_df["Metadata_Batch"].astype(str)
df_meta["Metadata_Batch"] = df_meta["Metadata_Batch"].astype(str)
images_df["Metadata_Plate"] = images_df["Metadata_Plate"].astype(str)
df_meta["Metadata_Plate"] = df_meta["Metadata_Plate"].astype(str)
images_df["Metadata_Source"] = images_df["Metadata_Source"].astype(str)
df_meta["Metadata_Source"] = df_meta["Metadata_Source"].astype(str)
images_df["Metadata_Well"] = images_df["Metadata_Well"].astype(str)
df_meta["Metadata_Well"] = df_meta["Metadata_Well"].astype(str)

merged_df = pd.merge(images_df, df_meta, on=["Metadata_Source", "Metadata_Plate", "Metadata_Well"])
merged_df

In [57]:
len(merged_df)

5121200

In [61]:
merged_df[(merged_df['Metadata_PlateType']=='COMPOUND')]["Metadata_JCP2022"].nunique()

112482

In [62]:
df_meta[(df_meta['Metadata_PlateType']=='COMPOUND')]["Metadata_JCP2022"].nunique()

115733

In [63]:
(115733-112482)/115733*100

2.8090518693890245

In [68]:
key_columns = ['Metadata_Source', 'Metadata_Plate', 'Metadata_Well']

df1 = merged_df[merged_df['Metadata_PlateType']=='COMPOUND'][['Metadata_Source', 'Metadata_Well', 'Metadata_Plate', 'Metadata_JCP2022']]
df2 = df_meta[(df_meta['Metadata_PlateType']=='COMPOUND')]
only_in_df1 = df1.merge(df2, on=key_columns, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)
only_in_df2 = df2.merge(df1, on=key_columns, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)

In [80]:
only_in_df2[only_in_df2['Metadata_Source']!='source_7']['Metadata_Plate'].unique()

array(['Dest210803-160702', 'EC000157'], dtype=object)

In [95]:
key_columns = ['Metadata_Source', 'Metadata_Plate', 'Metadata_Well']

df1 = merged_df[merged_df['Metadata_PlateType']!='COMPOUND'][['Metadata_Source', 'Metadata_Well', 'Metadata_Plate', 'Metadata_JCP2022']]
df2 = df_meta[(df_meta['Metadata_PlateType'].isin(['TARGET2', 'CRISPR', 'ORF', 'TARGET1']))]
only_in_df1 = df1.merge(df2, on=key_columns, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)
only_in_df2 = df2.merge(df1, on=key_columns, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)

In [99]:
only_in_df2['Metadata_Source'].value_counts()

Metadata_Source
source_4    9442
source_7    2688
Name: count, dtype: int64

In [96]:
only_in_df2

,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_PlateType,Metadata_Well,Metadata_JCP2022_x,Metadata_InChIKey,Metadata_InChI,Metadata_broad_sample,Metadata_Name,...,Metadata_Microscope_Name,Metadata_Widefield_vs_Confocal,Metadata_Excitation_Type,Metadata_Objective_NA,Metadata_N_Brightfield_Planes_Min,Metadata_N_Brightfield_Planes_Max,Metadata_Distance_Between_Z_Microns,Metadata_Sites_Per_Well,Metadata_Filter_Configuration,Metadata_JCP2022_y
418995,source_4,2021_04_26_Batch1,BR00117035,ORF,B15,JCP2022_999999,NaN,NaN,NaN,NaN,...,Opera Phenix,Widefield,Laser,1.00,3,3,5.0,9,B,NaN
418996,source_4,2021_04_26_Batch1,BR00117035,ORF,B16,JCP2022_999999,NaN,NaN,NaN,NaN,...,Opera Phenix,Widefield,Laser,1.00,3,3,5.0,9,B,NaN
419300,source_4,2021_04_26_Batch1,BR00117035,ORF,D20,JCP2022_999999,NaN,NaN,NaN,NaN,...,Opera Phenix,Widefield,Laser,1.00,3,3,5.0,9,B,NaN
419301,source_4,2021_04_26_Batch1,BR00117035,ORF,D21,JCP2022_999999,NaN,NaN,NaN,NaN,...,Opera Phenix,Widefield,Laser,1.00,3,3,5.0,9,B,NaN
419326,source_4,2021_04_26_Batch1,BR00117035,ORF,E02,JCP2022_900001,NaN,NaN,BAD CONSTRUCT,BAD CONSTRUCT,...,Opera Phenix,Widefield,Laser,1.00,3,3,5.0,9,B,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084853,source_7,20211211_Run9M,CP3-SC2-25M,TARGET2,P20,JCP2022_032357,HSUGRBWQSSZJOP-UHFFFAOYSA-N,InChI=1S/C22H26N2O4S/c1-15(25)28-20-21(16-9-11...,NaN,NaN,...,CV7000,Confocal,Laser,0.75,0,0,NaN,9,D,NaN
1084854,source_7,20211211_Run9M,CP3-SC2-25M,TARGET2,P21,JCP2022_047545,KXBDTLQSDKGAEB-UHFFFAOYSA-N,InChI=1S/C22H22FN5O3/c1-3-20(29)25-16-5-4-6-17...,NaN,NaN,...,CV7000,Confocal,Laser,0.75,0,0,NaN,9,D,NaN
1084855,source_7,20211211_Run9M,CP3-SC2-25M,TARGET2,P22,JCP2022_043099,JZFPYUNJRRFVQU-UHFFFAOYSA-N,"InChI=1S/C13H9F3N2O2/c14-13(15,16)8-3-1-4-9(7-...",NaN,NaN,...,CV7000,Confocal,Laser,0.75,0,0,NaN,9,D,NaN
1084856,source_7,20211211_Run9M,CP3-SC2-25M,TARGET2,P23,JCP2022_050516,LNFZRMDSZJCZTG-UHFFFAOYSA-N,InChI=1S/C28H21F4N5O2/c1-15-4-6-19(35-27(39)37...,NaN,NaN,...,CV7000,Confocal,Laser,0.75,0,0,NaN,9,D,NaN


In [105]:
merged_df.columns

Index(['Metadata_Source', 'Metadata_Batch_x', 'Metadata_Plate',
       'Metadata_Well', 'Metadata_Site', 'FileName_OrigAGP',
       'FileName_OrigDNA', 'FileName_OrigER', 'FileName_OrigMito',
       'FileName_OrigRNA', 'Metadata_Batch_y', 'Metadata_PlateType',
       'Metadata_JCP2022', 'Metadata_InChIKey', 'Metadata_InChI',
       'Metadata_broad_sample', 'Metadata_Name', 'Metadata_Vector',
       'Metadata_Transcript', 'Metadata_Symbol_x', 'Metadata_NCBI_Gene_ID_x',
       'Metadata_Taxon_ID', 'Metadata_Gene_Description', 'Metadata_Prot_Match',
       'Metadata_Insert_Length', 'Metadata_pert_type',
       'Metadata_NCBI_Gene_ID_y', 'Metadata_Symbol_y',
       'Metadata_Microscope_Name', 'Metadata_Widefield_vs_Confocal',
       'Metadata_Excitation_Type', 'Metadata_Objective_NA',
       'Metadata_N_Brightfield_Planes_Min',
       'Metadata_N_Brightfield_Planes_Max',
       'Metadata_Distance_Between_Z_Microns', 'Metadata_Sites_Per_Well',
       'Metadata_Filter_Configuration'],
      

In [113]:
df = merged_df[merged_df["Metadata_PlateType"]=='COMPOUND'][['Metadata_Source', 'Metadata_Batch_x', 'Metadata_Plate',
       'Metadata_Well', 'Metadata_JCP2022']]
df.columns = ['Metadata_Source', 'Metadata_Batch', 'Metadata_Plate', 'Metadata_Well', 'Metadata_JCP2022']

In [151]:
ref_compound_df = df[~df['Metadata_JCP2022'].isin(['JCP2022_999999', 'JCP2022_UNKNOWN'])].drop_duplicates()[~df['Metadata_JCP2022'].isin(['JCP2022_999999', 'JCP2022_UNKNOWN'])].reset_index().drop(columns='index')

/tmp/ipykernel_3390879/1714066172.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ref_compound_df = df[~df['Metadata_JCP2022'].isin(['JCP2022_999999', 'JCP2022_UNKNOWN'])].drop_duplicates()[~df['Metadata_JCP2022'].isin(['JCP2022_999999', 'JCP2022_UNKNOWN'])].reset_index().drop(columns='index')


In [152]:
ref_compound_df

,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022
0,source_10,2021_05_31_U2OS_48_hr_run1,Dest210531-152149,A01,JCP2022_085227
1,source_10,2021_05_31_U2OS_48_hr_run1,Dest210531-152149,A02,JCP2022_033924
2,source_10,2021_05_31_U2OS_48_hr_run1,Dest210531-152149,A03,JCP2022_056163
3,source_10,2021_05_31_U2OS_48_hr_run1,Dest210531-152149,A04,JCP2022_054175
4,source_10,2021_05_31_U2OS_48_hr_run1,Dest210531-152149,A06,JCP2022_008983
...,...,...,...,...,...
746870,source_9,20211103-Run16,GR00004421,Z24,JCP2022_037716
746871,source_9,20211103-Run16,GR00004421,Z25,JCP2022_037716
746872,source_9,20211103-Run16,GR00004421,Z26,JCP2022_033924
746873,source_9,20211103-Run16,GR00004421,Z47,JCP2022_033924


## Counts

### Resnet

In [121]:
len(ref_compound_df)

785137

In [122]:
len(df_resnet)

746875

In [153]:

df2 = df_resnet.copy(deep=True)

key_columns = ['Metadata_Source', 'Metadata_Plate', 'Metadata_Well']

only_in_ref = ref_compound_df.merge(df2, on=key_columns, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)
only_in_df2 = df2.merge(ref_compound_df, on=key_columns, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)



In [154]:
print(len(only_in_df2))
print(len(only_in_ref))

0
0


### chada

In [155]:
len(df_chada)

746870

In [158]:

df2 = df_chada.copy(deep=True)

key_columns = ['Metadata_Source', 'Metadata_Plate', 'Metadata_Well']

only_in_ref = ref_compound_df.merge(df2, on=key_columns, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)
only_in_df2 = df2.merge(ref_compound_df, on=key_columns, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)

print(len(only_in_df2))
print(len(only_in_ref))

0
5


In [159]:
only_in_ref # One molecule that is strange in formula so should have failed while preprocessed by @ThomasLemercier

,Metadata_Source,Metadata_Batch_x,Metadata_Plate,Metadata_Well,Metadata_JCP2022_x,Metadata_InChI,Metadata_PlateType,Metadata_Batch_y,Metadata_Is_Control,Embeddings_mean,Metadata_Row,Metadata_Col,Path_Embeddings,Metadata_JCP2022_y
273751,source_2,20210823_Batch_10,1086291931,A08,JCP2022_065740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
295607,source_3,CP59,BR5871b3,A15,JCP2022_065740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
367421,source_3,CP_35_all_Phenix1,BAY5871b,A15,JCP2022_065740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
501659,source_6,p211012CPU2OS48hw384exp033JUMP,110000295620,A15,JCP2022_065740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
579163,source_8,J3,A1170530,A15,JCP2022_065740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### dino (s and g have the exact same lines)

In [168]:
len(df_dinov2_s.drop_duplicates(subset=key_columns))

992234

In [171]:
duplicates = df_dinov2_g.groupby(key_columns).filter(lambda x: len(x) > 1)


In [172]:
duplicates

,Metadata_Well,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_JCP2022,Metadata_InChI,Metadata_Is_Control,Metadata_Row,Metadata_Col,path_embedding
25526,G23,source_6,p211109CPU2OS48hw384exp035JUMP,110000295498,nan,nan,False,7,23,/projects/synsight/data/dinov2/raw_well_embedd...
167155,N11,source_2,20210712_Batch_5,1053597844,nan,nan,False,14,11,/projects/synsight/data/dinov2/raw_well_embedd...
345067,G23,source_6,p211109CPU2OS48hw384exp035JUMP,110000295498,JCP2022_033924,InChI=1S/C2H6OS/c1-4(2)3/h1-2H3,True,7,23,/projects/synsight/data/dinov2/raw_well_embedd...
444675,B07,source_2,20210823_Batch_10,1086291962,nan,nan,False,2,7,/projects/synsight/data/dinov2/raw_well_embedd...
626192,N11,source_2,20210712_Batch_5,1053597844,JCP2022_064669,InChI=1S/C14H10F2N4O/c15-11-3-2-10(5-12(11)16)...,False,14,11,/projects/synsight/data/dinov2/raw_well_embedd...
863785,B07,source_2,20210823_Batch_10,1086291962,JCP2022_UNKNOWN,nan,False,2,7,/projects/synsight/data/dinov2/raw_well_embedd...


In [161]:

df2 = df_dinov2_s.copy(deep=True)

key_columns = ['Metadata_Source', 'Metadata_Plate', 'Metadata_Well']

only_in_ref = ref_compound_df.merge(df2, on=key_columns, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)
only_in_df2 = df2.merge(ref_compound_df, on=key_columns, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)

print(len(only_in_df2))
print(len(only_in_ref))

245360
0


### OpenPhenom

In [162]:
len(df_phenom)

992234

In [163]:

df2 = df_phenom.copy(deep=True)

key_columns = ['Metadata_Source', 'Metadata_Plate', 'Metadata_Well']

only_in_ref = ref_compound_df.merge(df2, on=key_columns, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)
only_in_df2 = df2.merge(ref_compound_df, on=key_columns, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)

print(len(only_in_df2))
print(len(only_in_ref))

245359
0


In [ ]:

df2 = df_dinov2_s.copy(deep=True)

key_columns = ['Metadata_Source', 'Metadata_Plate', 'Metadata_Well']

only_in_phenom = df_phenom.merge(df2, on=key_columns, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)
only_in_df2 = df2.merge(df_phenom, on=key_columns, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)

print(len(only_in_df2))
print(len(only_in_phenom))

0
0
